In [4]:
import sys
sys.path.append('../')

import networkx as nx
from node2vec import Node2Vec

from src.utils import load_embeddings

%load_ext autoreload
%autoreload 2

In [5]:
G = nx.read_gpickle('data/network.gpickle')
G.number_of_nodes(), G.number_of_edges()

(60789, 75647)

To search for node embeddings I use node2vec approach. I tried to use network SVD from network-sklearn, but it didn't work well (I got very low scaled vectors with values about 1e-20). node2vec gives more pleasant embeddings in terms of using dot product or cosine similarity. By the way it's not so fast (model trains for 3,5 minutes for graph with 60k vertices and 75k edges with some hyperparameters tuning)

In [6]:
def generate_node_embeddings(graph: nx.Graph):
    model = Node2Vec(graph, dimensions=128, num_walks=25, walk_length=25, temp_folder='temp/', workers=3)
    result = model.fit(window=5, min_count=1)
    return result

In [8]:
%%time
embeddings = generate_node_embeddings(G)

Computing transition probabilities:   0%|          | 0/60789 [00:00<?, ?it/s]

Generating walks (CPU: 1):   0%|          | 0/9 [00:00<?, ?it/s]

Generating walks (CPU: 1): 100%|██████████| 9/9 [02:05<00:00, 13.94s/it]


CPU times: user 25min 26s, sys: 3.15 s, total: 25min 29s
Wall time: 10min 3s


In [9]:
for idx, e in enumerate(G.edges()):
    if idx == 5:
        break
    print(e)

(0, 1)
(0, 24)
(0, 350192)
(1, 2)
(2, 3)


In [10]:
embeddings.wv.most_similar('0')

[('1', 0.9891837239265442),
 ('350192', 0.9847670793533325),
 ('24', 0.9673959612846375),
 ('2', 0.9611027240753174),
 ('350191', 0.950224757194519),
 ('200921', 0.9415721893310547),
 ('3', 0.9173770546913147),
 ('200923', 0.9080461263656616),
 ('350190', 0.8951256275177002),
 ('332388', 0.8791016936302185)]

In [12]:
embeddings.save('models_data/embeddings128.data')

In [ ]:
# join embeddings with corresponding nodes (as x feature)
for idx, node in enumerate(G.nodes()):
    G.add_node(int(node), x=embeddings.wv[str(node)].copy())

In [ ]:
nx.readwrite.write_gpickle(G, 'data/network128.gpickle')